In [ ]:
import os
import pandas as pd

from draft_optimizer.src.platform.espn import League
from draft_optimizer.src.utils import DATA_DIR

In [ ]:
# Get league
league_id = 88497130
year = 2022
league = League(id=league_id, year=year)

In [ ]:
# Get pro teams and schedule
pro_teams, pro_schedule = league.get_pro_schedule()

# Convert to df
pro_teams_df = pd.DataFrame.from_dict([v.dict() for v in pro_teams.values()]).sort_values("id").reset_index(drop=True)
pro_schedule_df = pd.DataFrame.from_dict([v2.dict() for v1 in pro_schedule.values() for v2 in v1]).sort_values(["week", "date", "home_id", "away_id"]).reset_index(drop=True)

In [ ]:
# Get players
# max_players = 25
max_players = None
players = league.get_players(max_players=max_players)

# Convert to df
players_df = pd.DataFrame.from_dict([v.dict() for v in players.values()]).sort_values("id").reset_index(drop=True)

In [ ]:
# Export data
league_dir = os.path.join(DATA_DIR, f"espn_{league_id}_{year}")
if not os.path.isdir(league_dir):
    os.mkdir(league_dir)
pro_teams_df.to_csv(os.path.join(league_dir, "pro_teams.csv"), index=False)
pro_schedule_df.to_csv(os.path.join(league_dir, "pro_schedule.csv"), index=False)
players_df.to_csv(os.path.join(league_dir, "pro_players.csv"), index=False)